Go and understand this: https://docs.geoserver.org/stable/en/user/security/passwd.html

And this: http://www.jasypt.org/encrypting-passwords.html

note: When using a random salt generator, two digests created for the same message will always be different (except in the case of random salt coincidence or no usage of salt). Because of this, in this case the result of the digest method will contains both the undigested salt and the digest of salt+message, so that another digest operation can be performed with the same salt on a different message to check if both messages match (all of which will be managed automatically by the matches method).



Take a known encrypted password from a geosever users.xml, e.g. 

```D9miJH/hVgfxZJscMafEtbtliG0ROxhLfsznyWfG38X2pda2JOSV4POi55PQI4tw```

This is the result of hashing the standard admin password "geoserver" and a random 16 byte salt for 100.000 iterations and then prepending the last hash with the undigested salt.

From these instructions we can in principle build a geoserver password generator that will produce a valid admin password at container startup to be copied into the geoserver data dir.

Let's first try and find out whether we can produce the encrypted password by extracting the first 16 bytes and performing the hashing operation.


In [3]:
import hashlib
import base64

# Known inputs: Password and Hash
password = "geoserver"
pw_hash = "D9miJH/hVgfxZJscMafEtbtliG0ROxhLfsznyWfG38X2pda2JOSV4POi55PQI4tw"

def prepend_and_hash(password, pw_hash):
    """
    This function takes a known password and its known hash as stored in the 
    data directory and then reverse engineers the hashing procedure as out-
    lined in the jasypt documentation using the parameters from the kartoza
    geoserver docker file.
    """
    # Decode the Base64 encoded string
    decoded_bytes = base64.b64decode(pw_hash)
    first_16_bytes = decoded_bytes[:16]

    # Prepend the first 16 bytes to the input string
    # The first 16 bytes are the random salt used for
    # obfuscating the password as such.
    prepended_string = first_16_bytes + password.encode()

    # Perform 100,000 iterations of SHA-256 hashing
    sha256_hash = prepended_string
    for _ in range(100000):
        sha256_hash = hashlib.sha256(sha256_hash).digest()

    # Prepend the first 16 bytes to the final hash
    # in their undigested form to allow password matching
    final_hash = first_16_bytes + sha256_hash

    # Convert the final hash to a text string
    base64_encoded = base64.b64encode(final_hash)

    output_string = base64_encoded.decode()

    return output_string



hashed_value = prepend_and_hash(password, pw_hash)
if hashed_value == pw_hash:
    print("Success. This works.")
else:
    print("Go home and think again.")



Success. This works.
